In [ ]:
import numpy as np
import scipy
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set()

# Read the data and inspect it

In [ ]:
df = pd.read_csv("../data/flights/2019-01-flights.csv.zip")

In [ ]:
df.head()

- Look at dep time. This is a strange format. We need to parse it.
- Implement a parser for this format. We should write tests, but that is beyond scope for this tutorial.

In [ ]:
df.dtypes

In [ ]:
# Why is DEP_TIME a float64? Is it sometimes nan?
len(df[df['DEP_TIME'].isna()])

In [ ]:
# How many nan times do we have if we ignore cancelled and diverted flights?
tdf = df[(df['CANCELLED'] == 0) & (df['DIVERTED'] == 0)]
len(tdf[tdf['DEP_TIME'].isna()]), len(tdf[tdf['ARR_TIME'].isna()])

In [ ]:
# Let's ignore the cancelled and diverted flights for the moment
df = df[(df['CANCELLED'] == 0) & (df['DIVERTED'] == 0)]

# Fix Table Format

## Convert types of time fields and drop irrelevant fields

In [ ]:
# Let's convert the time fields to int64 to make conversion to time simpler and drop fields we do not care about
df = df.astype({'DEP_TIME': 'int64', 'ARR_TIME': 'int64'}).drop(['CANCELLED', 'CANCELLATION_CODE', 'DIVERTED'], 1)
df.head()

## Convert FL_DATE to datetime

In [ ]:
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])
df.head()

## Convert CRS_*x*_TIME fields to time fields

In [ ]:
import datetime
def int_to_time(inttime):
    hours = inttime // 100
    mins = inttime % 100
    return datetime.time(hours, mins)
# Should write tests here
int_to_time(100), int_to_time(1133), int_to_time(2356)

In [ ]:
# Convert int times to times
df['CRS_DEP_TIME'] = df['CRS_DEP_TIME'].apply(int_to_time)

In [ ]:
df['DEP_TIME'].apply(int_to_time).head()
# whoops -- this fails

In [ ]:
import datetime
def int_to_time(inttime):
    hours = inttime // 100 % 24
    mins = inttime % 100
    return datetime.time(hours, mins)
# Should write tests here
int_to_time(100), int_to_time(1133), int_to_time(2356), int_to_time(2405)

In [ ]:
df['DEP_TIME'] = df['DEP_TIME'].apply(int_to_time)
df['CRS_ARR_TIME'] = df['CRS_ARR_TIME'].apply(int_to_time)
df['ARR_TIME'] = df['ARR_TIME'].apply(int_to_time)

In [ ]:
df.head()

# Compute Delay

In [ ]:
def row_to_delay(row):
    row_date = row['FL_DATE']
    planned_arrival = row['CRS_ARR_TIME']
    arrival = row['ARR_TIME']
    diff = datetime.datetime(row_date.year, row_date.month, row_date.day, 
                             arrival.hour, arrival.minute) - \
        datetime.datetime(row_date.year, row_date.month, row_date.day, 
                          planned_arrival.hour, planned_arrival.minute)
    return diff
df['delay_minutes'] = df.apply(row_to_delay, 1).apply(lambda x: x.total_seconds() / 60)

In [ ]:
df.head()

# Let's save the result

In [ ]:
analysis_columns = ['FL_DATE', 'OP_CARRIER', 'ORIGIN', 'DEST', 'delay_minutes']
df[analysis_columns].to_csv("../data/output/2019-01-flights-preprocessed.csv")